In [ ]:
%%capture
from google.colab import drive

drive.mount("/content/drive")

%cd /content/drive/MyDrive/nyu-stuff/2023-fall/DS-UA\ 301\ Advanced\ DS/final_project

%pip install -r requirements.txt
%pip install wandb -Uq
import pandas
import numpy
import json5
import functools
import json
from matplotlib import pyplot
from typing import Optional, Any, Coroutine, Union, Dict, List
import nltk
from nltk.corpus import stopwords
import string
import contractions
import torch
import wandb
import sklearn
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

nltk.download("averaged_perceptron_tagger")
nltk.download("universal_tagset")
nltk.download("stopwords")
nltk.download("punkt")

def get_best_torch_device():
    if torch.cuda.is_available():  # CUDA
        return "cuda"
    elif torch.backends.mps.is_available():  # Apple Silicon / GPU
        return "mps"
    else:  # Fallback to CPU
        return "cpu"

In [ ]:
# Configure hyperparameters (+ random state)
random_state = 13117

device = get_best_torch_device()
print(f"Currently using {device.upper()} for training!")

In [ ]:
%ls data/

amazon_reviews.zip                  checkpoints/                      sweep_result_loss_mp.pickle
balanced_data.csv                   cleaned_data.csv                  train_results.csv
balanced_with_labels.csv            labels.csv                        wandb_runs.csv
Cell_Phones_and_Accessories_5.json  sweep_result_loss_mp_fast.pickle


In [ ]:
trained_models = torch.load("data/sweep_result_loss_mp_fast.pickle")

In [ ]:
len(trained_models)

17

In [ ]:
trained_models_sorted = dict(sorted(trained_models.items()))

In [24]:
print("Here are the losses from trained combinations in increasing order:\n", "\n\t"
.join([]
      + list(
          [str(key) for key in trained_models_sorted.keys()]
          )
      )
)

Here are the losses from trained combinations in increasing order:
 0.2101428359746933
	0.2785088121891022
	0.3324443995952606
	0.3562460243701935
	0.36414608359336853
	0.39766794443130493
	0.4855074882507324
	0.5044924020767212
	0.5837665796279907
	0.5856112241744995
	0.9738161563873291
	0.994530439376831
	1.028523564338684
	1.0527478456497192
	1.2106595039367676
	1.43212890625
	1.5313397645950317


In [27]:
keys = list(trained_models_sorted.keys())
props_best = trained_models[keys[0]]

In [32]:
config = props_best["chkpt"].keys()

In [33]:
config

dict_keys(['model_state_dict', 'optimizer_state_dict', 'epoch', 'loss', 'val_loss', 'val_accuracy'])

In [29]:
for k, v in props_best.items():

    # Let's not dump the model just yet
    if k == "result_df":
        pyplot.plot(df_results["train_loss"], label="train_loss")
        pyplot.plot(df_results["val_loss"], label="val_loss")
        pyplot.legend()
        pyplot.title(f"Losses vs Number of Epochs")
        continue
    elif k == "chkpt":
      continue





dict_keys(['result_df', 'val_loss', 'val_acc', 'test_acc', 'chkpt'])

In [ ]:
proms